# Flood Height to Raster Processor

This code takes flood heights and DEMs as inputs and uses ArcPy tools to find the inundated areas for each division and output them as a binary raster of flooded or not flooded for each division and the entire area of interest.

## Imports

In [1]:
import arcpy
from arcpy import env
from arcpy.sa import *
import pandas as pd
import os

## Flood Height CSV

CSV file that contains flood heights from running the Flood Estimate code.

In [2]:
flood_heights = pd.read_csv("Sample_Output_Data\\single_storm_flood_heights_c.csv",header=None).values[0]
flood_heights

array([1.96195622, 1.17523596, 1.96195622, 0.331069  , 0.8522816 ,
       0.        , 1.96195622, 1.96195622, 1.96195622, 1.96195622,
       0.        , 0.        , 1.08288026, 1.19047646, 0.19457727,
       0.81034823, 1.16141369, 0.86217069])

## Create raster

Converts flood heights into raster and vector files that can be viewed on a GIS software.
1. Uses the ArcPy Con function to set areas where the flood height is lower than the elevation of the DEM to 0, and where flood height is higher to 1 for each division. (Produces raster files in the IntermediateData folder)
2. Combines each division into one large raster using the ArcPy Mosaic to Raster function. (Produces one raster file)
3. Uses the ArcPy Set Null function to set 0 values in the raster to NoData values.
4. Uses the ArcPy Raster to Polygon function to convert the areas where flood height is higher than the DEM into polygons.

In [3]:
#name of flood raster
flood_area_name = "inundation_area"

# location of original, divisioned DEM files
env.workspace = r"Data\LM_div18"
env.overwriteOutput = True

rasters=[]

#Initializing the folders if not created
divfolder = r"IntermediateData\div_Rasters" #intermediate data is not needed for the final result
if not os.path.exists(divfolder):
    os.makedirs(divfolder)
rasterfolder = r"Sample_Output_Data\Flood_Rasters"
if not os.path.exists(rasterfolder):
    os.makedirs(rasterfolder)

#Creating a raster for each division
for i in range(len(flood_heights)):
    raster_name=r"Data\LM_div18\dem_lm_z35_%d.TIF"%i
    inRaster = Raster(raster_name)
    whereClause="VALUE<%f"%flood_heights[i]
    outCon = Con(inRaster, 1, 0, whereClause) #using the ArcGIS function "Con"
    outPath = r"IntermediateData\div_Rasters\storm_div%d.TIF"%i
    outCon.save(outPath)
    rasters.append(outPath)

#Combining all rasters together to encompass the whole study area
raster_mosaic = arcpy.MosaicToNewRaster_management(rasters, rasterfolder , flood_area_name+".tif",None,None,None,1,None,None)

#turn raster to polygon
env.workspace = rasterfolder
env.overwriteOutput = True
outSetNull = SetNull(raster_mosaic, raster_mosaic, "VALUE=0")
arcpy.RasterToPolygon_conversion(outSetNull, out_polygon_features=flood_area_name +".shp", simplify="NO_SIMPLIFY", raster_field="Value")


<Result 'Sample_Output_Data\\Flood_Rasters\\inundation_area.shp'>